In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df_train = pd.read_csv(r'train.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [4]:
from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler()


x_input = df_train.drop(columns=['label']).values
x = mm_scaler.fit_transform(x_input).reshape(-1, 28, 28)
y = pd.get_dummies(df_train['label'])

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [12]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers import BatchNormalization

from keras.layers.convolutional import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='tanh'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(x_train, y_train, batch_size=32, epochs=15, validation_data=(x_test, y_test))

Epoch 1/15


2022-08-21 17:31:41.715097: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 105369600 exceeds 10% of free system memory.


1050/1050 [==============================] - 20s 19ms/step - loss: 0.2483 - accuracy: 0.9254 - val_loss: 0.0812 - val_accuracy: 0.9754
Epoch 2/15
1050/1050 [==============================] - 21s 20ms/step - loss: 0.0814 - accuracy: 0.9755 - val_loss: 0.0638 - val_accuracy: 0.9806
Epoch 3/15
1050/1050 [==============================] - 20s 19ms/step - loss: 0.0613 - accuracy: 0.9803 - val_loss: 0.0457 - val_accuracy: 0.9857
Epoch 4/15
1050/1050 [==============================] - 21s 20ms/step - loss: 0.0479 - accuracy: 0.9845 - val_loss: 0.0395 - val_accuracy: 0.9889
Epoch 5/15
1050/1050 [==============================] - 24s 23ms/step - loss: 0.0408 - accuracy: 0.9870 - val_loss: 0.0405 - val_accuracy: 0.9880
Epoch 6/15
1050/1050 [==============================] - 21s 20ms/step - loss: 0.0336 - accuracy: 0.9896 - val_loss: 0.0414 - val_accuracy: 0.9883
Epoch 7/15
1050/1050 [==============================] - 21s 20ms/step - loss: 0.0301 - accuracy: 0.9901 - val_loss: 0.0424 - val_accura

In [ ]:
df_test = pd.read_csv('test.csv')

x_out = df_test.values
x_output = mm_scaler.fit_transform(x_input).reshape(-1, 28, 28)
y_pred = pd.get_dummies(df_test)


x_out = mm_scaler.transform(df_test.values)
x_out_aux = x_out.reshape(-1, 28, 28)
x_output = np.hstack([x_out, v_out, h_out])
y_pred = np.argmax(model.predict(x_output), axis=1).reshape(-1, 1)